# Retrievers

A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

Retrievers accept a string query as input and return a list of Document's as output.

Retrievers are Runnable´s, it can be chained automatically then.

In [5]:
from langchain_community.chat_models.ollama import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain_experimental.text_splitter import SemanticChunker

#local_llm = ChatOlloma(model="phi3", base_url="http://127.0.0.1:11434", temperature=0)
local_llm = ChatOpenAI(model="gpt-4o", temperature=0)
embedding_function = OpenAIEmbeddings(model="text-embedding-3-large")
#embedding_function = OllamaEmbeddings(model="nomic-embed-text")

# FAISS is an in-memory vector store
def semantic_chunking(documents: list[Document], threshold=25) -> list[Document]:
    text_splitter = SemanticChunker(
    embeddings=embedding_function, 
    breakpoint_threshold_type="percentile", # "percentile" "standard_deviation", "interquartile",
    breakpoint_threshold_amount=threshold
    )
    return text_splitter.split_documents(documents)

def create_in_memory_vector_store(chunks:list[Document]) -> FAISS:
    # First ensure we have valid documents
    if not chunks:
        raise ValueError("No documents provided")
    
    # Convert documents to texts and metadatas
    texts = [doc.page_content for doc in chunks]
    metadatas = [doc.metadata for doc in chunks]
    
    # Create FAISS index from texts
    return FAISS.from_texts(
        texts=texts,
        embedding=embedding_function,
        metadatas=metadatas
    )    

def load_pdf(file_path:str) -> list[Document]:
    from langchain_community.document_loaders.pdf import PyPDFLoader
    loader_PDF = PyPDFLoader(file_path=file_path, extract_images=True)
    docsPdf = loader_PDF.load_and_split()
    return docsPdf

## The simple retriever

In [3]:
from rich import print
file_path = "../Resources/The Code Whisperer.pdf"
documents = load_pdf(file_path)
print(documents)

[
    Document(
        page_content='**The Code Whisperer**  \nMeet Alex, a brilliant software developer with a passion for 
crafting elegant code. By day, Alex \nworked at TechCorp, a leading fintech firm in Silicon Valley. By night, Alex 
built innovative \napps in his tiny home office, fueled by copious amounts of coffee and elec tronic dance \nmusic.
\nAlex\'s colleagues revered him as "The Code Whisperer." His ability to debug complex issues \nand architect 
scalable solutions earned him a reputation as a coding genius. His code was \nalways clean, efficient, and 
beautifully commented – a rare breed in the so ftware \ndevelopment world.  \nOne fateful evening, Alex received an
intriguing email from a mysterious startup, NeuroSpark. \nThey needed a lead developer to build an AI -powered 
trading platform, capable of analyzing \nmarket trends and making predictions with uncanny accuracy. The project 
\'s ambition and \npotential for disruption excited Alex. He agreed to meet the founders, Rachel and Jake, at a 
\ntrendy café in San Francisco.  \nOver artisanal coffee, Rachel and Jake revealed their vision: to democratize 
access to AI -\ndriven investment insights, making the financial markets more inclusive and transparent. \nAlex was
sold. He joined NeuroSpark as the lead developer, assembling a team of talented \nengineers to bring the platform 
to life.  \nThe journey was not without its challenges. Alex\'s team battled against tight deadlines, \nByzantine 
regulatory hurdles, and the occasional all -nighter. However, their collective \npassion and Alex\'s technical 
mastery prevailed. The platform, dubbed "Pulse," b egan to take \nshape.  \nPulse\'s launch was met with widespread
acclaim. Financial institutions, hedge funds, and \nindividual investors clamored to integrate the AI -driven 
insights into their operations. Alex\'s \ncreation had transformed the trading landscape, earning him recognition  
as one of the most \ninfluential software developers of the year.  \nAs Alex looked out over the San Francisco 
skyline, he smiled, knowing that his code had \nchanged the game. The Code Whisperer had whispered his magic once 
more, and the world \nwas a better place for it.',
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    )
]

In [6]:
chunks = semantic_chunking(documents, threshold=20)
print(chunks)
#Expected execution time: 8s

[
    Document(
        page_content='**The Code Whisperer**  \nMeet Alex, a brilliant software developer with a passion for 
crafting elegant code. By day, Alex \nworked at TechCorp, a leading fintech firm in Silicon Valley. By night, Alex 
built innovative \napps in his tiny home office, fueled by copious amounts of coffee and elec tronic dance \nmusic.
Alex\'s colleagues revered him as "The Code Whisperer." His ability to debug complex issues \nand architect 
scalable solutions earned him a reputation as a coding genius.',
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content='His code was \nalways clean, efficient, and beautifully commented – a rare breed in the so 
ftware \ndevelopment world.',
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content='One fateful evening, Alex received an intriguing email from a mysterious startup, 
NeuroSpark.',
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content='They needed a lead developer to build an AI -powered trading platform, capable of analyzing 
\nmarket trends and making predictions with uncanny accuracy.',
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content="The project 's ambition and \npotential for disruption excited Alex.",
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content='He agreed to meet the founders, Rachel and Jake, at a \ntrendy café in San Francisco. Over 
artisanal coffee, Rachel and Jake revealed their vision: to democratize access to AI -\ndriven investment insights,
making the financial markets more inclusive and transparent.',
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content='Alex was sold.',
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content='He joined NeuroSpark as the lead developer, assembling a team of talented \nengineers to 
bring the platform to life.',
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content='The journey was not without its challenges.',
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content="Alex's team battled against tight deadlines, \nByzantine regulatory hurdles, and the 
occasional all -nighter.",
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content="However, their collective \npassion and Alex's technical mastery prevailed.",
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content='The platform, dubbed "Pulse," b egan to take \nshape.',
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content="Pulse's launch was met with widespread acclaim.",
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content='Financial institutions, hedge funds, and \nindividual investors clamored to integrate the AI 
-driven insights into their operations.',
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content="Alex's \ncreation had transformed the trading landscape, earning him recognition  as one of 
the most \ninfluential software developers of the year.",
        metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
    ),
    Document(
        page_content='As Alex looked out over the San Francisco skyline, he smiled, knowing that his code had 
\nchanged the game.',
        metadata={'source': '../Resources/The Code Whispe

In [7]:

vectorstore = create_in_memory_vector_store(chunks)

In [8]:
#Remember the similarity search ?
results = vectorstore.similarity_search_with_relevance_scores("Who are the founders?", k=3)
#Remember.. this has nothing to do with AI.. this is just a vector distance where the vectors are embedded
#vectorized similar sentences
print(results)

/home/darthlinuxer/AI/venv/lib/python3.12/site-packages/langchain_core/vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='He agreed to meet the founders, Rachel and Jake, at a \ntrendy café in San Francisco. Over artisanal coffee, Rachel and Jake revealed their vision: to democratize access to AI -\ndriven investment insights, making the financial markets more inclusive and transparent.', metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}), 0.11370622954417464), (Document(page_content='He joined NeuroSpark as the lead developer, assembling a team of talented \nengineers to bring the platform to life.', metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}), 0.01969616665407914), (Document(page_content="Alex's \ncreation had transformed the trading landscape, earning him recognition  as one of the most \ninfluential software developers of the year.", metadata={'source': '../Resources/The Code Whispe

[
    (
        Document(
            page_content='He agreed to meet the founders, Rachel and Jake, at a \ntrendy café in San Francisco. 
Over artisanal coffee, Rachel and Jake revealed their vision: to democratize access to AI -\ndriven investment 
insights, making the financial markets more inclusive and transparent.',
            metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
        ),
        0.11370622954417464
    ),
    (
        Document(
            page_content='He joined NeuroSpark as the lead developer, assembling a team of talented \nengineers to 
bring the platform to life.',
            metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
        ),
        0.01969616665407914
    ),
    (
        Document(
            page_content="Alex's \ncreation had transformed the trading landscape, earning him recognition  as one 
of the most \ninfluential software developers of the year.",
            metadata={'source': '../Resources/The Code Whisperer.pdf', 'page': 0}
        ),
        -0.07875760706191404
    )
]

In [10]:
#Retrievers are exactly like similarity search.. but can be used in a chain.
from langchain_core.runnables import RunnablePassthrough # This just says: The parameter will be provided later
from langchain_core.output_parsers import StrOutputParser # gets rid of the AI Output boilerplate coming from the LLM
from langchain_core.prompts import (ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate)
                                    
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("""
    You are a helpful assistant that give an aswer only based on the provided contexts: 
    ============================================================================
    Contexts: \n -> {context}
    ============================================================================"""),
    HumanMessagePromptTemplate.from_template("Question: {question}")
]) #yes you don´t need to remember this syntax anymore: [{'role':'system','content':system}, {'role':'user','content...}

simple_retriever = vectorstore.as_retriever(
    search_function="similarity", # Can be "similarity" (default), "mmr", or "similarity_score_threshold"
    search_kwargs={"k":5})  #k=5 returns the 5 most relevant chunks in the vector database

def format_document(documents:list[Document])->str:
    return "\n -> ".join([f"{i}:{doc.page_content}" for i,doc in enumerate(documents)])

chain = (
    {"context":simple_retriever | format_document, "question": RunnablePassthrough()}
    | prompt 
    #| local_llm
    #| StrOutputParser()
)

result = chain.invoke("Who are the founders?") #there is only one RunnablePassTrough(), you can type the question directly
print(result)
print(result.dict()['messages'][0]['content'])

ChatPromptValue(
    messages=[
        SystemMessage(
            content='\n    You are a helpful assistant that give an aswer only based on the provided contexts: \n  
============================================================================\n    Contexts: \n -> 0:He agreed to 
meet the founders, Rachel and Jake, at a \ntrendy café in San Francisco. Over artisanal coffee, Rachel and Jake 
revealed their vision: to democratize access to AI -\ndriven investment insights, making the financial markets more
inclusive and transparent.\n -> 1:He joined NeuroSpark as the lead developer, assembling a team of talented 
\nengineers to bring the platform to life.\n -> 2:Alex\'s \ncreation had transformed the trading landscape, earning
him recognition  as one of the most \ninfluential software developers of the year.\n -> 3:One fateful evening, Alex
received an intriguing email from a mysterious startup, NeuroSpark.\n -> 4:The platform, dubbed "Pulse," b egan to 
take \nshape.\n    ============================================================================'
        ),
        HumanMessage(content='Question: Who are the founders?')
    ]
)

You are a helpful assistant that give an aswer only based on the provided contexts: 
    ============================================================================
    Contexts: 
 -> 0:He agreed to meet the founders, Rachel and Jake, at a 
trendy café in San Francisco. Over artisanal coffee, Rachel and Jake revealed their vision: to democratize access 
to AI -
driven investment insights, making the financial markets more inclusive and transparent.
 -> 1:He joined NeuroSpark as the lead developer, assembling a team of talented 
engineers to bring the platform to life.
 -> 2:Alex's 
creation had transformed the trading landscape, earning him recognition  as one of the most 
influential software developers of the year.
 -> 3:One fateful evening, Alex received an intriguing email from a mysterious startup, NeuroSpark.
 -> 4:The platform, dubbed "Pulse," b egan to take 
shape.
    ============================================================================

## MultiQuery Retriever

Distance-based vector database retrieval embeds (represents) queries in high-dimensional space and finds similar embedded documents based on “distance”. But, retrieval may produce different results with subtle changes in query wording or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.

The MultiQueryRetriever automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the MultiQueryRetriever might be able to overcome some of the limitations of the distance-based retrieval and get a richer set of results.

In [11]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=simple_retriever,
    llm=local_llm
)

import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

results = await multiquery_retriever.ainvoke("Who are the founders?")
print([f"\n - {i}: {doc.page_content}" for i,doc in enumerate(results)])

INFO:langchain.retrievers.multi_query:Generated queries: ['Who established the organization?  ', 'Who initiated the founding of the company?  ', 'Who were the original creators?']
/home/darthlinuxer/AI/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.aget_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use ainvoke instead.
  warn_deprecated(


[
    '\n - 0: He agreed to meet the founders, Rachel and Jake, at a \ntrendy café in San Francisco. Over artisanal 
coffee, Rachel and Jake revealed their vision: to democratize access to AI -\ndriven investment insights, making 
the financial markets more inclusive and transparent.',
    '\n - 1: He joined NeuroSpark as the lead developer, assembling a team of talented \nengineers to bring the 
platform to life.',
    '\n - 2: The platform, dubbed "Pulse," b egan to take \nshape.',
    '\n - 3: One fateful evening, Alex received an intriguing email from a mysterious startup, NeuroSpark.',
    "\n - 4: The project 's ambition and \npotential for disruption excited Alex.",
    "\n - 5: Alex's \ncreation had transformed the trading landscape, earning him recognition  as one of the most 
\ninfluential software developers of the year.",
    '\n - 6: **The Code Whisperer**  \nMeet Alex, a brilliant software developer with a passion for crafting 
elegant code. By day, Alex \nworked at TechCorp, a leading fintech firm in Silicon Valley. By night, Alex built 
innovative \napps in his tiny home office, fueled by copious amounts of coffee and elec tronic dance \nmusic. 
Alex\'s colleagues revered him as "The Code Whisperer." His ability to debug complex issues \nand architect 
scalable solutions earned him a reputation as a coding genius.'
]

## Filters

### LLMCHainFilter

Uses LLM chain to decide which of the initially retrieved documents to filter out and which ones to return, without manipulating the document contents.

In [12]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(
    local_llm
)  # but this call to LLM slows down the process
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=multiquery_retriever
)

results = compression_retriever.invoke("Who are the founders?")
print([f"\n - {i}: {doc.page_content}" for i, doc in enumerate(results)])

# Expected execution time 42s

INFO:langchain.retrievers.multi_query:Generated queries: ['Who established the organization?  ', 'Who initiated the founding of the company?  ', 'Who were the original creators?']
/home/darthlinuxer/AI/venv/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/venv/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/venv/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/venv/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLM

[
    '\n - 0: He agreed to meet the founders, Rachel and Jake, at a \ntrendy café in San Francisco. Over artisanal 
coffee, Rachel and Jake revealed their vision: to democratize access to AI -\ndriven investment insights, making 
the financial markets more inclusive and transparent.'
]

## Multiple strategies and Document Transforms together

Using the DocumentCompressorPipeline we can also easily combine multiple strategies in sequence. Along with different compressors and filters we can add DocumentTransformers to our pipeline, which don’t perform any contextual compression but simply perform some transformation on a set of documents. For example Text Splitters can be used as document transformers to split documents into smaller pieces, and the EmbeddingsRedundantFilter can be used to filter out redundant documents based on embedding similarity between documents.

Below we create a compressor pipeline by first splitting our docs into smaller chunks, then removing redundant documents, and then filtering based on relevance to the query.

In [13]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import (DocumentCompressorPipeline, EmbeddingsFilter, LLMChainFilter)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_transformers import (EmbeddingsRedundantFilter, LongContextReorder)

# Split the chunks even more
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0, separators=[". "])

#Remove the redundant documents by using an EmbeddingsRedundantFilter
redundant_filter = EmbeddingsRedundantFilter(embeddings=embedding_function)

# The EmbeddingsFilter # provides a cheaper and faster option by embedding the documents and 
# return only the ones with sufficiently similar embeddings to the query.
relevant_filter = EmbeddingsFilter(embeddings= embedding_function, similarity_threshold=0.2)

reorder_filter = LongContextReorder()

llm_filter = LLMChainFilter.from_llm(local_llm) #THIS ONE USES LLM to check similarity, it will slow the process

pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter, reorder_filter, llm_filter]
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=multiquery_retriever
)

compressed_docs = compression_retriever.invoke(
    "Who are the founders?"
)

for i,doc in enumerate(compressed_docs):
    print(f"- {i}: {doc.page_content}")
    
# Expected execution time: 24s (without llm filter) 41s with (llm filter)

INFO:langchain.retrievers.multi_query:Generated queries: ['Who established the organization?  ', 'Who initiated the founding of the company?  ', 'Who were the original creators?']
/home/darthlinuxer/AI/venv/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/venv/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/venv/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/venv/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLM

- 0: He agreed to meet the founders, Rachel and Jake, at a 
trendy café in San Francisco

- 1: . Over artisanal coffee, Rachel and Jake revealed their vision: to democratize access to AI -
driven investment insights, making the financial markets more inclusive and transparent.